In [1]:
# No data or code to analyze yet
print("No data or key variables available in Finished Tasks section")

No data or key variables available in Finished Tasks section


In [2]:
import os
import json
import uuid
import datetime
from pathlib import Path
import shutil

class FileSystemStorage:
    """
    A file-based JSON storage system for the multi-agent website/app builder.
    This class handles creating, reading, updating, and versioning project files.
    """
    
    def __init__(self, base_path="./data/projects"):
        """
        Initialize the storage service with a base path for storing project files.
        
        Args:
            base_path (str): The base directory path for storing project files
        """
        self.base_path = Path(base_path)
        self.versions_path = self.base_path / "versions"
        self.initialize()
    
    def initialize(self):
        """Create necessary directories if they don't exist."""
        self.base_path.mkdir(parents=True, exist_ok=True)
        self.versions_path.mkdir(parents=True, exist_ok=True)
    
    def create_project(self, user_prompt):
        """
        Create a new project based on a user prompt.
        
        Args:
            user_prompt (str): The user's description of the project
            
        Returns:
            str: The newly created project ID
        """
        project_id = str(uuid.uuid4())
        timestamp = datetime.datetime.now().isoformat()
        
        # Create initial project structure
        project_data = {
            "projectId": project_id,
            "name": f"Project {project_id[:8]}",
            "description": user_prompt,
            "createdAt": timestamp,
            "updatedAt": timestamp,
            "files": {},
            "versions": [],
            "currentAgentIndex": 0,
            "agentStatuses": [
                {
                    "name": "emma",
                    "role": "Product Manager",
                    "status": "pending",
                    "completedAt": None
                },
                {
                    "name": "bob",
                    "role": "Architect",
                    "status": "pending",
                    "completedAt": None
                },
                {
                    "name": "alex",
                    "role": "Engineer",
                    "status": "pending",
                    "completedAt": None
                },
                {
                    "name": "david",
                    "role": "Data Analyst",
                    "status": "pending",
                    "completedAt": None
                },
                {
                    "name": "mike",
                    "role": "Team Leader",
                    "status": "pending",
                    "completedAt": None
                }
            ]
        }
        
        # Save the project file
        project_file_path = self.base_path / f"{project_id}.json"
        with open(project_file_path, "w") as f:
            json.dump(project_data, f, indent=2)
        
        # Create first version snapshot (initial state)
        version_id = f"v0-{timestamp.replace(':', '-')}"
        self.create_version_snapshot(project_id, version_id, "Project initialized", project_data)
        
        return project_id
    
    def get_project(self, project_id):
        """
        Retrieve a project by its ID.
        
        Args:
            project_id (str): The ID of the project to retrieve
            
        Returns:
            dict: The project data or None if not found
        """
        project_file_path = self.base_path / f"{project_id}.json"
        if not project_file_path.exists():
            return None
        
        with open(project_file_path, "r") as f:
            return json.load(f)
    
    def save_project(self, project_id, project_data):
        """
        Save updated project data.
        
        Args:
            project_id (str): The ID of the project
            project_data (dict): The updated project data
            
        Returns:
            bool: True if successful, False otherwise
        """
        project_data["updatedAt"] = datetime.datetime.now().isoformat()
        project_file_path = self.base_path / f"{project_id}.json"
        
        with open(project_file_path, "w") as f:
            json.dump(project_data, f, indent=2)
        
        return True
    
    def list_projects(self):
        """
        List all projects.
        
        Returns:
            list: A list of project IDs
        """
        project_files = list(self.base_path.glob("*.json"))
        return [p.stem for p in project_files]
    
    def get_file_content(self, project_id, file_path):
        """
        Get the content of a file within a project.
        
        Args:
            project_id (str): The ID of the project
            file_path (str): The path of the file within the project
            
        Returns:
            dict: File data including content, lastModified, and createdBy
        """
        project_data = self.get_project(project_id)
        if not project_data or file_path not in project_data["files"]:
            return None
        
        return project_data["files"][file_path]
    
    def create_or_update_file(self, project_id, file_path, content, agent):
        """
        Create or update a file within a project.
        
        Args:
            project_id (str): The ID of the project
            file_path (str): The path of the file within the project
            content (str): The content of the file
            agent (str): The name of the agent or "User" creating/updating the file
            
        Returns:
            bool: True if successful, False otherwise
        """
        project_data = self.get_project(project_id)
        if not project_data:
            return False
        
        timestamp = datetime.datetime.now().isoformat()
        
        # Check if file already exists
        file_exists = file_path in project_data["files"]
        
        # Create or update the file
        project_data["files"][file_path] = {
            "content": content,
            "lastModified": timestamp,
            "createdBy": project_data["files"].get(file_path, {}).get("createdBy", agent) if file_exists else agent
        }
        
        # Save the updated project
        self.save_project(project_id, project_data)
        
        return True
    
    def list_files(self, project_id):
        """
        List all files in a project.
        
        Args:
            project_id (str): The ID of the project
            
        Returns:
            list: A list of file paths
        """
        project_data = self.get_project(project_id)
        if not project_data:
            return []
        
        return list(project_data["files"].keys())
    
    def create_version_snapshot(self, project_id, version_id, description, project_data=None):
        """
        Create a version snapshot of the project.
        
        Args:
            project_id (str): The ID of the project
            version_id (str): The ID of the version
            description (str): Description of the version
            project_data (dict): The project data to save, or None to load current
            
        Returns:
            str: The ID of the created version
        """
        if project_data is None:
            project_data = self.get_project(project_id)
            if not project_data:
                return None
        
        timestamp = datetime.datetime.now().isoformat()
        
        # Create a copy of the project data for this version
        version_data = {
            "id": version_id,
            "timestamp": timestamp,
            "description": description,
            "files": project_data["files"].copy()
        }
        
        # Add version to project record
        if version_id not in [v["id"] for v in project_data.get("versions", [])]:
            project_data["versions"].append({
                "id": version_id,
                "timestamp": timestamp,
                "description": description
            })
            self.save_project(project_id, project_data)
        
        # Save version snapshot to versions directory
        version_file_path = self.versions_path / f"{project_id}-{version_id}.json"
        with open(version_file_path, "w") as f:
            json.dump(version_data, f, indent=2)
        
        return version_id
    
    def get_version(self, project_id, version_id):
        """
        Get a specific version of a project.
        
        Args:
            project_id (str): The ID of the project
            version_id (str): The ID of the version
            
        Returns:
            dict: The version data or None if not found
        """
        version_file_path = self.versions_path / f"{project_id}-{version_id}.json"
        if not version_file_path.exists():
            return None
        
        with open(version_file_path, "r") as f:
            return json.load(f)
    
    def list_versions(self, project_id):
        """
        List all versions of a project.
        
        Args:
            project_id (str): The ID of the project
            
        Returns:
            list: A list of version objects
        """
        project_data = self.get_project(project_id)
        if not project_data or "versions" not in project_data:
            return []
        
        return project_data["versions"]
    
    def restore_version(self, project_id, version_id):
        """
        Restore a project to a specific version.
        
        Args:
            project_id (str): The ID of the project
            version_id (str): The ID of the version to restore
            
        Returns:
            bool: True if successful, False otherwise
        """
        # Get the version data
        version_data = self.get_version(project_id, version_id)
        if not version_data:
            return False
        
        # Get current project data
        project_data = self.get_project(project_id)
        if not project_data:
            return False
        
        # Update project files with version files
        project_data["files"] = version_data["files"]
        project_data["updatedAt"] = datetime.datetime.now().isoformat()
        
        # Save the restored project
        self.save_project(project_id, project_data)
        
        # Create a new version snapshot marking the restoration
        new_version_id = f"restore-{version_id}-{datetime.datetime.now().isoformat().replace(':', '-')}"
        description = f"Restored from version {version_id}"
        self.create_version_snapshot(project_id, new_version_id, description, project_data)
        
        return True
    
    def update_agent_status(self, project_id, agent_name, status, files_created=None):
        """
        Update the status of an agent in a project.
        
        Args:
            project_id (str): The ID of the project
            agent_name (str): The name of the agent
            status (str): The new status (e.g., "completed", "in_progress", "pending")
            files_created (list): List of files created or modified by the agent
            
        Returns:
            bool: True if successful, False otherwise
        """
        project_data = self.get_project(project_id)
        if not project_data:
            return False
        
        timestamp = datetime.datetime.now().isoformat()
        
        # Find and update the agent status
        for agent in project_data["agentStatuses"]:
            if agent["name"] == agent_name:
                agent["status"] = status
                if status == "completed":
                    agent["completedAt"] = timestamp
                break
        
        # Move to next agent if current one completed
        if status == "completed":
            next_index = project_data["currentAgentIndex"] + 1
            if next_index < len(project_data["agentStatuses"]):
                project_data["currentAgentIndex"] = next_index
        
        # Save the updated project
        self.save_project(project_id, project_data)
        
        # Create a version snapshot for this agent completion
        if status == "completed" and agent_name:
            version_id = f"{agent_name}-{timestamp.replace(':', '-')}"
            description = f"{agent_name.capitalize()} completed work"
            if files_created:
                description += f" on {len(files_created)} files"
            self.create_version_snapshot(project_id, version_id, description, project_data)
        
        return True


# Example usage of the storage system
if __name__ == "__main__":
    # Create a storage instance
    storage = FileSystemStorage()
    
    # Create a new project
    project_id = storage.create_project("Create a React + Firebase chat app with login and admin dashboard")
    print(f"Created project with ID: {project_id}")
    
    # Create some files in the project
    storage.create_or_update_file(
        project_id, 
        "docs/PRD.md", 
        "# Product Requirements Document\n\n## Overview\nA React + Firebase chat application with user authentication and admin dashboard.",
        "emma"
    )
    
    # Update agent status
    storage.update_agent_status(project_id, "emma", "completed", ["docs/PRD.md"])
    
    # Create more files as different agents
    storage.create_or_update_file(
        project_id,
        "docs/Architecture.md",
        "# Architecture Document\n\n## Tech Stack\n- React\n- Firebase\n- Material UI",
        "bob"
    )
    storage.update_agent_status(project_id, "bob", "completed", ["docs/Architecture.md"])
    
    # List all files in the project
    files = storage.list_files(project_id)
    print(f"Project files: {files}")
    
    # List all versions
    versions = storage.list_versions(project_id)
    print(f"Project versions: {versions}")
    
    # Example of restoring a version
    if versions:
        first_version = versions[0]["id"]
        storage.restore_version(project_id, first_version)
        print(f"Restored project to version: {first_version}")

Created project with ID: 6a2d8f2c-5aa0-4725-b727-762c1af5d5b5
Project files: ['docs/PRD.md', 'docs/Architecture.md']
Project versions: [{'id': 'v0-2025-06-02T05-30-35.919667', 'timestamp': '2025-06-02T05:30:35.930399', 'description': 'Project initialized'}, {'id': 'emma-2025-06-02T05-30-35.971829', 'timestamp': '2025-06-02T05:30:36.008065', 'description': 'Emma completed work on 1 files'}, {'id': 'bob-2025-06-02T05-30-36.067401', 'timestamp': '2025-06-02T05:30:36.079589', 'description': 'Bob completed work on 1 files'}]
Restored project to version: v0-2025-06-02T05-30-35.919667
